In [14]:
from exchange.log import Trade
from exchange.log import TradeBar

In [15]:
loader = Trade()

loader.append('../DATA/BB/05/BB-2020-05-09T06-16-23.367761Z.log.gz')
loader.append('../DATA/BB/05/BB-2020-05-09T06-17-17.496249Z.log.gz')

print(loader.log_data.head())
print(loader.end_time)

../DATA/BB/05/BB-2020-05-09T06-16-23.367761Z.log.gz 2020050906-0522--2020050906-0533
../DATA/BB/05/BB-2020-05-09T06-17-17.496249Z.log.gz 2020050906-0516--2020050907-0526
   action                       time  sequence    price  volume  checksum
0       1 2020-05-09 06:16:22.636752         0      NaN     NaN       NaN
1       3 2020-05-09 06:16:22.636752         0  96020.0   0.025       NaN
2       3 2020-05-09 06:16:22.636752         1  96050.0   0.045       NaN
3       3 2020-05-09 06:16:22.636752         2  96060.0   4.100       NaN
4       3 2020-05-09 06:16:22.636752         3  96065.0   0.025       NaN
2020-05-09 07:10:26.235445


In [16]:
loader.update_logs()

loader.exec_log

,action,time,sequence,price,volume,checksum
410,6,2020-05-09 06:16:17.152,0,98195.0,0.009,NaN
411,4,2020-05-09 06:16:22.732,0,98200.0,0.002,NaN
412,4,2020-05-09 06:16:23.737,0,98200.0,0.017,NaN
489,4,2020-05-09 06:16:32.158,0,98200.0,0.016,NaN
560,4,2020-05-09 06:16:47.168,0,98200.0,0.472,NaN
...,...,...,...,...,...,...
71261,6,2020-05-09 07:10:03.501,0,97285.0,0.066,NaN
71316,6,2020-05-09 07:10:07.191,0,97285.0,1.696,NaN
71366,4,2020-05-09 07:10:13.858,0,97290.0,0.002,NaN
71380,6,2020-05-09 07:10:15.309,0,97285.0,0.100,NaN


In [17]:
bar = TradeBar()

bar.setup_ochlv(loader)


print(bar.bar['open'])
print(bar.bar['close'])
print(bar.bar['high'])
print(bar.bar['low'])

time
0      98195.0
1      98200.0
2      98200.0
3      98200.0
4      98200.0
        ...   
271    97330.0
272    97325.0
273    97285.0
274    97290.0
275    97305.0
Name: open, Length: 276, dtype: float64
time
0      98195.0
1      98200.0
2      98200.0
3      98200.0
4      98200.0
        ...   
271    97325.0
272    97295.0
273    97285.0
274    97285.0
275    97305.0
Name: close, Length: 276, dtype: float64
time
0      98195.0
1      98200.0
2      98200.0
3      98200.0
4      98200.0
        ...   
271    97330.0
272    97325.0
273    97285.0
274    97290.0
275    97305.0
Name: high, Length: 276, dtype: float64
time
0      98195.0
1      98200.0
2      98200.0
3      98200.0
4      98200.0
        ...   
271    97325.0
272    97295.0
273    97285.0
274    97285.0
275    97305.0
Name: low, Length: 276, dtype: float64


In [ ]:
bar.bar

In [ ]:
bar.setup_dollar_bar(loader, 2)

bar.bar


In [20]:
import numpy as np

df = loader.exec_log.copy()
df['sum'] = df['volume'].cumsum()  # 1BTC for each bin
df['sum'] = np.floor(df['sum']/2)
df

,action,time,sequence,price,volume,checksum,sum
410,6,2020-05-09 06:16:17.152,0,98195.0,0.009,NaN,0.0
411,4,2020-05-09 06:16:22.732,0,98200.0,0.002,NaN,0.0
412,4,2020-05-09 06:16:23.737,0,98200.0,0.017,NaN,0.0
489,4,2020-05-09 06:16:32.158,0,98200.0,0.016,NaN,0.0
560,4,2020-05-09 06:16:47.168,0,98200.0,0.472,NaN,0.0
...,...,...,...,...,...,...,...
71261,6,2020-05-09 07:10:03.501,0,97285.0,0.066,NaN,867.0
71316,6,2020-05-09 07:10:07.191,0,97285.0,1.696,NaN,868.0
71366,4,2020-05-09 07:10:13.858,0,97290.0,0.002,NaN,868.0
71380,6,2020-05-09 07:10:15.309,0,97285.0,0.100,NaN,868.0


In [21]:
df.loc[df['action'].isin([4, 5]), 'sell_volume'] = df['volume']
df.loc[df['action'].isin([6, 7]), 'buy_volume'] = df['volume']

df = df.groupby('sum').agg({'time': 'last', 'price': ['first', 'last', 'max', 'min'],
                                   'sell_volume': 'sum', 'buy_volume': 'sum'}, axis=1)
df.columns = ['time_stamp', 'open', 'close', 'high', 'low', 'sell_volume', 'buy_volume']
df.reset_index(drop=True, inplace=True)
df.index.name = 'time'
df = df[['time_stamp', 'open', 'close', 'high', 'low', 'sell_volume', 'buy_volume']]
df




,time_stamp,open,close,high,low,sell_volume,buy_volume
time,,,,,,,
0,2020-05-09 06:17:04.177,98195.0,98200.0,98200.0,98195.0,1.303,0.009
1,2020-05-09 06:19:17.098,98200.0,98155.0,98200.0,98155.0,2.301,0.306
2,2020-05-09 06:19:33.977,98140.0,98080.0,98140.0,98080.0,0.000,0.235
3,2020-05-09 06:19:33.977,98080.0,98070.0,98080.0,98070.0,0.000,7.012
4,2020-05-09 06:19:33.977,98055.0,98055.0,98055.0,98055.0,0.000,1.140
...,...,...,...,...,...,...,...
608,2020-05-09 07:09:28.188,97355.0,97325.0,97360.0,97325.0,1.010,1.833
609,2020-05-09 07:09:57.129,97330.0,97300.0,97330.0,97300.0,0.091,1.514
610,2020-05-09 07:10:03.501,97295.0,97285.0,97295.0,97285.0,0.000,1.544


In [22]:
bar.update_price()

chop too short 2020-05-09 07:10:16.309000 2020-05-09 07:13:16.309000
chop too short 2020-05-09 07:10:24.224000 2020-05-09 07:13:24.224000
chop too short 2020-05-09 07:10:24.224000 2020-05-09 07:13:24.224000


In [23]:
bar.bar

,time_stamp,open,close,high,low,sell_volume,buy_volume,bs_ratio,market_buy,market_sell,limit_buy,limit_sell
time,,,,,,,,,,,,
0,2020-05-09 06:17:04.177,98195.0,98200.0,98200.0,98195.0,0.009,1.303,0.993140,NaN,98195.0,98195.0,98200.0
1,2020-05-09 06:19:17.098,98200.0,98155.0,98200.0,98155.0,0.306,2.301,0.882624,NaN,98140.0,98100.0,98160.0
2,2020-05-09 06:19:33.977,98140.0,98080.0,98140.0,98080.0,0.235,0.000,0.000000,NaN,98080.0,98080.0,98085.0
3,2020-05-09 06:19:33.977,98080.0,98070.0,98080.0,98070.0,7.012,0.000,0.000000,NaN,98080.0,98080.0,98085.0
4,2020-05-09 06:19:33.977,98055.0,98055.0,98055.0,98055.0,1.140,0.000,0.000000,NaN,98080.0,98080.0,98085.0
...,...,...,...,...,...,...,...,...,...,...,...,...
608,2020-05-09 07:09:28.188,97355.0,97325.0,97360.0,97325.0,1.833,1.010,0.355259,NaN,97325.0,97325.0,97330.0
609,2020-05-09 07:09:57.129,97330.0,97300.0,97330.0,97300.0,1.514,0.091,0.056698,NaN,NaN,97295.0,97305.0
610,2020-05-09 07:10:03.501,97295.0,97285.0,97295.0,97285.0,1.544,0.000,0.000000,97290.0,NaN,97285.0,97290.0
